In [1]:
import pandas as pd
from custom_func import *

In [2]:
dict_regions = {'Вінницька': 'Вінницька ОДА',
 'Волинська': 'Волинська ОДА',
 'Дніпропетровська': 'Дніпропетровська ОДА',
 'Донецька': 'Донецька ОДА',
 'Житомирська': 'Житомирська ОДА',
 'Закарпатська': 'Закарпатська ОДА',
 'Запорізька': 'Запорізька ОДА',
 'Івано-Франківська': 'Івано-Франківська ОДА',
 'м. Київ': 'Київська МДА',
 'Київська': 'Київська ОДА',
 'Кіровоградська': 'Кіровоградська ОДА',
 'Луганська': 'Луганська ОДА',
 'Львівська': 'Львівська ОДА',
 'Миколаївська': 'Миколаївська ОДА',
 'Одеська': 'Одеська ОДА',
 'Полтавська': 'Полтавська ОДА',
 'Рівненська': 'Рівненська ОДА',
 'Сумська': 'Сумська ОДА',
 'Тернопільська': 'Тернопільська ОДА',
 'Харківська': 'Харківська ОДА',
 'Херсонська': 'Херсонська ОДА',
 'Хмельницька': 'Хмельницька ОДА',
 'Черкаська': 'Черкаська ОДА',
 'Чернівецька': 'Чернівецька ОДА',
 'Чернігівська': 'Чернігівська ОДА'}

# p1_01, p1_02

Мінімально змінював робочий для v0.2 версії код. 

In [3]:
df_P01_001 = pd.read_excel(f'{INPUTS_PATH}/P1/P01_001.xlsx')
df_P01_001["oda_beneficiary"] = df_P01_001["Бенефіціар"].str.contains("\w+\s+ОДА$", case=False, na=False).astype(int)
df_P01_001 = df_P01_001.loc[df_P01_001['oda_beneficiary']==1].drop("oda_beneficiary", 1)
df_P01_001["Кінець"] = pd.to_datetime(df_P01_001["Кінець"])
df_P01_001["Початок"] = pd.to_datetime(df_P01_001["Початок"])

In [4]:
# p1_01 актуальні проєкти визначаю як ті, що закінчуться після звітнього періоду
df_P01_001_oda = df_P01_001.loc[
    df_P01_001['Кінець']>=pd.to_datetime('2020-06-30')
].copy()

df_P01_001_rows = []
df_P01_001_columns = ['region','quantity','sum_new_projects','sum_total']
for key, value in dict_regions.items():
    df_P01_001_rows.append(
        [
            key,
            df_P01_001_oda.loc[df_P01_001_oda['Бенефіціар'].str.contains(value)].shape[0], 
            # початок нового звітного періоду - 2020-01-01
            df_P01_001_oda.loc[
                (df_P01_001_oda['Початок']>=pd.to_datetime('2020-01-01')) & 
                (df_P01_001_oda["Початок"]<=pd.to_datetime('2020-06-30')) &
                (df_P01_001_oda['Бенефіціар'].str.contains(value))]['USD, \nтисяч'].sum()*1000, 
            df_P01_001_oda.loc[df_P01_001_oda['Бенефіціар'].str.contains(value)]['USD, \nтисяч'].sum()*1000
        ]
    )
    
    
df_P01_001_fin = pd.DataFrame(df_P01_001_rows,columns=df_P01_001_columns)
df_P01_001_fin["population"] = df_P01_001_fin["region"].map(dict_population)
df_P01_001_fin['sum_per_person'] = df_P01_001_fin['sum_total']/df_P01_001_fin['population']
df_P01_001_fin = df_P01_001_fin.loc[df_P01_001_fin["region"].ne("м. Київ")].copy()
normalize_parameter(df_P01_001_fin, 'sum_per_person', 'p1_01')
normalize_parameter(df_P01_001_fin, 'sum_new_projects', 'p1_02')

Емпіричні границі: (0.0, 25.127939271303827)
Коефіцієнти нормалізації: (0.039796339413395616, -0.0)
Параметр p1_01 нормалізовано і додано до таблиці

Емпіричні границі: (0.0, 878324.764650815)
Коефіцієнти нормалізації: (1.1385310311699544e-06, -0.0)
Параметр p1_02 нормалізовано і додано до таблиці



In [5]:
df_P01_002 = pd.read_excel(f'{INPUTS_PATH}/P1/P01_002.xlsx')
df_P01_002 = df_P01_002.loc[df_P01_002["region"].ne("м. Київ")].copy()
# p1_03
df_P01_002["p1_03_raw"] = df_P01_002["Видатки ДФРР"] / df_P01_002["Передбачено ДФРР"]

# p1_04
df_P01_002["p1_04_raw"] = df_P01_002["Передбачено ДФРР"] / df_P01_002["Затверджений обсяг фінансування"]

normalize_parameter(df_P01_002, "p1_03_raw", "p1_03",min_bound=0,max_bound=1)
normalize_parameter(df_P01_002, "p1_04_raw", "p1_04",min_bound=0,max_bound=1)

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p1_03 нормалізовано і додано до таблиці

Емпіричні границі: (0, 1)
Коефіцієнти нормалізації: (1.0, -0.0)
Параметр p1_04 нормалізовано і додано до таблиці



In [6]:
# # p1_03 - використання коштів визначаю через касові видатки

# # поки що всюди 0
# spent, total = (
#     "ДФРР_обсяг касових видатків наростаючим підсумком з початку року станом на останній день періоду",
#     "ДФРР_обсяг коштів, передбачений розподілом на відповідний рік за загальним та спеціальним фондом"
#     )

# df_KMU = pd.read_excel(f"{OUTPUT_PATH}/99_KMU/kmu.xlsx").rename(columns={"Область": "region"})
# df_KMU["Частка використаних коштів"] = df_KMU[spent] / df_KMU[total] * 100

# normalize_parameter(df_KMU, "Частка використаних коштів", "p1_03",min_bound=0,max_bound=1)

In [7]:
# # з новими проєктами виходить, що майже всюди частка коштів становить ~99%
# df_P01_002 = pd.read_excel(f'{OUTPUT_PATH}/P1/P01_002.xlsx')
# df_P01_002["ratio"] = df_P01_002["Затверджено КМУ_14.05"] / df_P01_002["Передбачено ДФРР 2020, тис. грн._30.04"]
# df_P01_002 = df_P01_002.loc[df_P01_002["region"].ne("м. Київ")].copy()
# normalize_parameter(df_P01_002, "ratio", "p1_04",min_bound=0,max_bound=1)

In [8]:
df_P1 = pd.DataFrame(df_P01_001_fin['region'])
P1_sources = [df_P01_001_fin, df_P01_002]
for source in P1_sources:
    df_P1 = df_P1.merge(source.loc[:, source.columns.str.contains("region$|p1.*\d+$")], on="region", how="outer")

In [9]:
dict_weights = {
    'p1_01':1,
    'p1_02':0.5,
    'p1_03':0.5,
    'p1_04':1,
}

# # p1_03 були nan (через нулі у чисельнику і знаменнику)
# df_P1["p1_03"].fillna(0, inplace=True)
# cols = df_P1.loc[:,df_P1.columns.str.contains('p')].columns
# # df_P1['P1'] = df_P1.loc[:,df_P1.columns.str.contains('p')].apply(lambda x: weighted_average(x,dict_weights,cols),axis=1)*10
# #df_P1['P1'] = df_P1.mean(axis=1)*10

sources = [df_P1]
save_data(sources,regions,dict_weights,'P1')

Дані за галузевим параметром P1 збережено
               region     p1_01     p1_02     p1_03     p1_04        P1
0           Волинська  1.000000  0.000000  0.229970  0.898782  6.712558
1           Вінницька  0.030567  0.000000  0.286085  0.465472  2.130272
2    Дніпропетровська  0.250081  0.000000  0.187103  0.607682  3.171047
3            Донецька  0.026206  0.000000  0.167278  0.726443  2.787628
4         Житомирська  0.048894  0.000000  0.211890  0.814445  3.230948
5        Закарпатська  0.839785  0.265229  0.129672  0.856026  6.310872
6          Запорізька  0.391019  0.000000  0.124642  0.637657  3.636655
7            Київська  0.052842  0.000000  0.242983  0.546132  2.401550
8   Івано-Франківська  0.487118  1.000000  0.160098  0.800366  6.225110
9      Кіровоградська  0.000000  0.000000  0.179614  0.880617  3.234746
10          Луганська  0.033150  0.000000  0.222960  0.861422  3.353509
11          Львівська  0.953718  0.040863  0.272911  0.773737  6.281142
12       Миколаївська 